In [37]:
#Estimating Quantity of Pizza Required to meet customer demands using time-series forecasting using Prophet
#Prophet is a robust open-source forecasting tool developed by Facebook designed to handle time
#series data with daily observations and strong seasonal effects. It is particularly useful for
#making forecasts that incorporate holidays, events, and other irregular time series components.
#Prophet is designed to work well with daily data, but it can be adapted for use with other
#frequencies, such as weekly data.

In [28]:
import pandas as pd
from collections import defaultdict

In [29]:
# Load the data
data = pd.read_csv('Updated_Pizza_Sales.csv')

data['order_date'] = pd.to_datetime(data['order_date'])

In [30]:
# Filter the data to keep only orders with date less than the 29th of each month, so we can split a month in exactly 4 weeks
filtered_data = data[data['order_date'].dt.day < 29]

# Filter the data to keep only the first 8 months
filtered_data = filtered_data[filtered_data['order_date'].dt.month <= 8]

In [31]:
# Initialize an empty dictionary to store the split data
split_data = {}

# Loop over each month (1 to 8)
for month in range(1, 9):
    # Filter data for the current month
    month_data = filtered_data[filtered_data['order_date'].dt.month == month]
    
    # Initialize a list to store the weekly data for the current month
    weekly_data = []
    
    # Loop over each week (1 to 4)
    for week in range(1, 5):
        # Calculate the start and end day for the current week
        start_day = (week - 1) * 7 + 1
        end_day = start_day + 6
        
        # Filter data for the current week
        week_data = month_data[(month_data['order_date'].dt.day >= start_day) & (month_data['order_date'].dt.day <= end_day)]
        
        # Append the weekly data to the list
        weekly_data.append(week_data)
    
    # Store the weekly data for the current month in the dictionary
    split_data[month] = weekly_data

# Verify the split data
for month, weeks in split_data.items():
    print(f"Month: {month}")
    for week_num, week_data in enumerate(weeks, start=1):
        print(f"  Week {week_num}: {week_data.shape[0]} rows")

Month: 1
  Week 1: 979 rows
  Week 2: 932 rows
  Week 3: 924 rows
  Week 4: 925 rows
Month: 2
  Week 1: 1038 rows
  Week 2: 954 rows
  Week 3: 949 rows
  Week 4: 951 rows
Month: 3
  Week 1: 938 rows
  Week 2: 952 rows
  Week 3: 984 rows
  Week 4: 891 rows
Month: 4
  Week 1: 998 rows
  Week 2: 944 rows
  Week 3: 946 rows
  Week 4: 909 rows
Month: 5
  Week 1: 920 rows
  Week 2: 1032 rows
  Week 3: 957 rows
  Week 4: 898 rows
Month: 6
  Week 1: 983 rows
  Week 2: 931 rows
  Week 3: 906 rows
  Week 4: 961 rows
Month: 7
  Week 1: 1068 rows
  Week 2: 907 rows
  Week 3: 985 rows
  Week 4: 952 rows
Month: 8
  Week 1: 909 rows
  Week 2: 1005 rows
  Week 3: 957 rows
  Week 4: 882 rows


In [32]:
# Initialize a dictionary to store aggregated data
aggregated_data = defaultdict(lambda: defaultdict(int))

# Loop over months 1 to 8
for month in range(1, 9):
    for week_num, week_data in enumerate(split_data[month], start=1):
        week_data_grouped = week_data.groupby('pizza_name')['quantity'].sum()
        for pizza_name, quantity in week_data_grouped.items():
            aggregated_data[(month, week_num)][pizza_name] += quantity

# Convert aggregated data to DataFrame for easier manipulation
aggregated_df = pd.DataFrame(aggregated_data).T.fillna(0)
aggregated_df.index.names = ['month', 'week']
aggregated_df.columns.name = 'pizza_name'

print(aggregated_df.head())


pizza_name  The Barbecue Chicken Pizza  The Big Meat Pizza  \
month week                                                   
1     1                             50                  37   
      2                             41                  30   
      3                             46                  34   
      4                             51                  37   
2     1                             55                  45   

pizza_name  The Brie Carre Pizza  The Calabrese Pizza  \
month week                                              
1     1                        8                   10   
      2                        5                   21   
      3                        7                   12   
      4                       12                   16   
2     1                       16                   31   

pizza_name  The California Chicken Pizza  The Chicken Alfredo Pizza  \
month week                                                            
1     1                

In [33]:
# Prepare data for Prophet
prophet_data = []

for (month, week), row in aggregated_df.iterrows():
    for pizza_name, quantity in row.items():
        # Create a proper datetime object for the start of each week
        year = 2020 if month <= 8 else 2021  # Assuming data is from 2020
        day = (week - 1) * 7 + 1
        date_str = f"{year}-{month:02d}-{day:02d}"
        prophet_data.append({
            'ds': date_str,
            'y': quantity,
            'pizza_name': pizza_name
        })

prophet_df = pd.DataFrame(prophet_data)
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

print(prophet_df.head())


          ds   y                    pizza_name
0 2020-01-01  50    The Barbecue Chicken Pizza
1 2020-01-01  37            The Big Meat Pizza
2 2020-01-01   8          The Brie Carre Pizza
3 2020-01-01  10           The Calabrese Pizza
4 2020-01-01  45  The California Chicken Pizza


In [34]:
from prophet import Prophet

# Initialize a dictionary to store forecasts
forecasts = {}

# Forecast for each pizza type
for pizza in aggregated_df.columns:
    # Prepare data for the specific pizza type
    pizza_data = prophet_df[prophet_df['pizza_name'] == pizza][['ds', 'y']]
    
    # Initialize and fit the Prophet model
    model = Prophet(weekly_seasonality=True, daily_seasonality=False)
    model.fit(pizza_data)
    
    # Create a dataframe for future dates (4 weeks in the 9th month)
    future_dates = model.make_future_dataframe(periods=4, freq='W-MON')
    forecast = model.predict(future_dates)
    
    # Store the forecast
    forecasts[pizza] = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    print(f"Forecast for {pizza}:\n", forecasts[pizza])


14:39:40 - cmdstanpy - INFO - Chain [1] start processing
14:39:41 - cmdstanpy - INFO - Chain [1] done processing


Forecast for The Barbecue Chicken Pizza:
            ds       yhat  yhat_lower  yhat_upper
0  2020-01-01  50.154299   43.786218   56.844890
1  2020-01-08  50.080430   43.456957   56.706111
2  2020-01-15  50.006562   43.205789   56.941884
3  2020-01-22  49.932694   42.743683   56.583554
4  2020-02-01  46.445787   39.619953   52.767766
5  2020-02-08  46.371918   39.889841   52.702990
6  2020-02-15  46.298050   39.896986   53.293064
7  2020-02-22  46.224182   39.354212   52.623356
8  2020-03-01  51.610368   44.788583   58.205276
9  2020-03-08  51.536499   45.299635   58.278562
10 2020-03-15  51.462631   44.942380   58.113295
11 2020-03-22  51.388762   44.733971   57.710892
12 2020-04-01  49.194010   42.718050   55.274194
13 2020-04-08  49.120141   42.259022   55.844158
14 2020-04-15  49.046273   42.792138   55.945189
15 2020-04-22  48.972404   42.176683   55.811546
16 2020-05-01  50.860179   43.670007   57.324039
17 2020-05-08  50.786311   44.026790   57.149467
18 2020-05-15  50.712442   

14:39:41 - cmdstanpy - INFO - Chain [1] start processing
14:39:41 - cmdstanpy - INFO - Chain [1] done processing
14:39:41 - cmdstanpy - INFO - Chain [1] start processing


Forecast for The Big Meat Pizza:
            ds       yhat  yhat_lower  yhat_upper
0  2020-01-01  34.199587   24.876044   42.688129
1  2020-01-08  34.312529   25.584685   43.377926
2  2020-01-15  34.425471   25.520681   42.825368
3  2020-01-22  34.538413   25.982400   43.300191
4  2020-02-01  36.153239   27.251641   45.415637
5  2020-02-08  36.266180   28.096336   45.045571
6  2020-02-15  36.379122   27.428547   44.617818
7  2020-02-22  36.492064   27.053598   45.169105
8  2020-03-01  37.845973   29.934090   46.651297
9  2020-03-08  37.958915   29.949726   46.828348
10 2020-03-15  38.071856   29.646351   47.566169
11 2020-03-22  38.184798   29.355788   47.722720
12 2020-04-01  35.667830   26.975398   44.273477
13 2020-04-08  35.780771   27.572847   45.065167
14 2020-04-15  35.893713   26.708957   43.939460
15 2020-04-22  36.006655   27.311848   44.258682
16 2020-05-01  40.888584   32.312736   50.003469
17 2020-05-08  41.001526   32.482025   50.112940
18 2020-05-15  41.114468   32.63562

14:39:41 - cmdstanpy - INFO - Chain [1] done processing
14:39:42 - cmdstanpy - INFO - Chain [1] start processing


...

Forecast for The Spinach and Feta Pizza:
            ds       yhat  yhat_lower  yhat_upper
0  2020-01-01  27.786655   20.638416   35.315869
1  2020-01-08  27.841624   20.855939   34.880912
2  2020-01-15  27.896594   21.099865   35.037321
3  2020-01-22  27.951563   21.103954   34.628318
4  2020-02-01  28.831751   22.118173   36.050535
5  2020-02-08  28.886721   21.582434   35.636740
6  2020-02-15  28.941690   21.946539   36.185258
7  2020-02-22  28.996660   22.868934   35.850855
8  2020-03-01  31.168001   24.433627   37.894868
9  2020-03-08  31.222971   24.212228   38.104575
10 2020-03-15  31.277940   24.361963   38.150774
11 2020-03-22  31.332909   24.511257   38.472702
12 2020-04-01  28.501258   22.025323   35.182670
13 2020-04-08  28.556227   21.365888   35.268428
14 2020-04-15  28.611197   21.806974   35.769600
15 2020-04-22  28.666166   22.249937   35.883447
16 2020-05-01  31.165877   24.471942   37.577377
17 2020-05-08  31.220847   24.366358   38.121248
18 2020-05-15  31.275816   

14:39:52 - cmdstanpy - INFO - Chain [1] done processing
14:39:53 - cmdstanpy - INFO - Chain [1] start processing


Forecast for The Thai Chicken Pizza:
            ds       yhat  yhat_lower  yhat_upper
0  2020-01-01  47.162498   39.595268   54.049496
1  2020-01-08  47.266226   40.159026   54.341927
2  2020-01-15  47.369953   40.027963   55.124068
3  2020-01-22  47.473681   41.038126   54.514231
4  2020-02-01  40.245645   33.114409   46.852707
5  2020-02-08  40.349372   33.466905   47.197820
6  2020-02-15  40.453100   33.224228   47.255231
7  2020-02-22  40.556828   33.465946   47.708782
8  2020-03-01  47.593912   40.278398   54.592230
9  2020-03-08  47.697640   40.308056   55.491572
10 2020-03-15  47.801368   40.635724   54.445543
11 2020-03-22  47.905096   41.389384   55.435801
12 2020-04-01  48.510959   41.741884   55.510383
13 2020-04-08  48.614687   41.633940   55.678939
14 2020-04-15  48.718415   41.390477   55.916803
15 2020-04-22  48.822143   42.293525   55.885481
16 2020-05-01  44.343945   37.318739   51.025398
17 2020-05-08  44.447673   37.376871   51.584340
18 2020-05-15  44.551401   37.5

14:39:53 - cmdstanpy - INFO - Chain [1] done processing


Forecast for The Vegetables + Vegetables Pizza:
            ds       yhat  yhat_lower  yhat_upper
0  2020-01-01  34.350279   29.923945   38.775446
1  2020-01-08  34.188191   29.980218   38.701153
2  2020-01-15  34.026102   29.749905   38.248486
3  2020-01-22  33.864013   29.543629   38.442324
4  2020-02-01  33.475337   28.950842   38.084020
5  2020-02-08  33.313208   28.989696   37.616024
6  2020-02-15  33.151078   28.745903   37.648631
7  2020-02-22  32.988949   28.386992   37.349512
8  2020-03-01  31.492686   27.033090   35.938313
9  2020-03-08  31.330556   27.255801   35.692214
10 2020-03-15  31.168427   26.719215   35.662198
11 2020-03-22  31.006298   26.447032   35.311663
12 2020-04-01  32.242720   27.641781   37.052143
13 2020-04-08  32.080590   27.656478   36.424831
14 2020-04-15  31.918461   27.229496   36.049492
15 2020-04-22  31.756332   27.769461   36.159791
16 2020-05-01  25.492969   21.212706   29.701727
17 2020-05-08  25.330840   20.753530   29.595167
18 2020-05-15  25.16

In [35]:
# Aggregate predictions for each week in the 9th month
predicted_quantities = defaultdict(dict)

for pizza, forecast in forecasts.items():
    for _, row in forecast.tail(4).iterrows():  # Only take the last 4 weeks (the 9th month)
        week = row['ds'].week
        predicted_quantities[week][pizza] = row['yhat']

# Convert the predictions to a DataFrame
predicted_df = pd.DataFrame(predicted_quantities).fillna(0).T
predicted_df.index.name = 'week'
predicted_df.columns.name = 'pizza_name'

print("Predicted quantities for the 9th month:\n", predicted_df)


Predicted quantities for the 9th month:
 pizza_name  The Barbecue Chicken Pizza  The Big Meat Pizza  \
week                                                         
35                           44.223875           33.947562   
36                           44.150006           34.060503   
37                           44.076138           34.173445   
38                           44.002270           34.286387   

pizza_name  The Brie Carre Pizza  The Calabrese Pizza  \
week                                                    
35                      9.732123            20.776054   
36                      9.753501            20.754784   
37                      9.774880            20.733513   
38                      9.796259            20.712243   

pizza_name  The California Chicken Pizza  The Chicken Alfredo Pizza  \
week                                                                  
35                             51.132295                  18.825968   
36                            